In [1]:
import numpy as np
import pandas as pd 
import random as rd
import sys
import io
import logging, os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import LambdaCallback

In [2]:
arquivo = pd.read_csv("Spotteds.csv")


arquivo_em_minusculo = arquivo.Documentos.str.lower()
minusculo_sem_acento = arquivo_em_minusculo.str.normalize('NFKD').str.encode('ascii', errors = 'ignore').str.decode('utf-8')
dados = minusculo_sem_acento.drop([17642, 17848, 17852, 17979, 18488, 24311])
spotteds_string = ''
for spotteds in dados:
  spotteds_string += spotteds


spotteds_set = set(spotteds_string)
spotteds_lista = sorted(list(spotteds_set))


# tipo a é 0, b é 1?? aham

dic = dict(enumerate(spotteds_lista))
print(dic)

dicionario_invertido = {v: k for k, v in dic.items()} # tem uma virgula. isso. VLw man

print(dicionario_invertido['f'])

{0: ' ', 1: '!', 2: '"', 3: '#', 4: '$', 5: '%', 6: '&', 7: "'", 8: '(', 9: ')', 10: '*', 11: '+', 12: ',', 13: '-', 14: '.', 15: '/', 16: '0', 17: '1', 18: '2', 19: '3', 20: '4', 21: '5', 22: '6', 23: '7', 24: '8', 25: '9', 26: ':', 27: ';', 28: '=', 29: '?', 30: '@', 31: 'M', 32: 'N', 33: 'T', 34: '[', 35: '\\', 36: ']', 37: '^', 38: '_', 39: 'a', 40: 'b', 41: 'c', 42: 'd', 43: 'e', 44: 'f', 45: 'g', 46: 'h', 47: 'i', 48: 'j', 49: 'k', 50: 'l', 51: 'm', 52: 'n', 53: 'o', 54: 'p', 55: 'q', 56: 'r', 57: 's', 58: 't', 59: 'u', 60: 'v', 61: 'w', 62: 'x', 63: 'y', 64: 'z', 65: '{', 66: '|', 67: '}', 68: '~'}
44


In [3]:
maxlen = 40
step = 3
entrada = []
saida = []
for i in dados:
  for j in range(0, len(i) - maxlen, step):
    entrada.append(i[j: j + maxlen])
    saida.append(i[j + maxlen])

entradaBi = []
saidaBi = []
for i in dados: #fazendo a leitura bilateral (de traz pra frente)
  for j in range(len(i), maxlen, -step):
    frase = i[j - maxlen: j]
    frase = frase[:: -1]
    entradaBi.append(frase) 
    saidaBi.append(i[j - maxlen - 1])

In [4]:
vetor_x = np.zeros((len(entrada), maxlen, len(spotteds_lista)), dtype=np.bool)
vetor_y = np.zeros((len(saida), len(spotteds_lista)), dtype=np.bool)
vetor_xBi = np.zeros((len(entradaBi), maxlen, len(spotteds_lista)), dtype=np.bool)
vetor_yBi = np.zeros((len(saidaBi), len(spotteds_lista)), dtype=np.bool)

for i in range(len(vetor_x)):# percorre as frases 
  for j in range(len(vetor_x[i])): #percorre cada caracter
    caracter = entrada[i][j]
    indice = dicionario_invertido[caracter]
    vetor_x[i][j][indice] = 1

for k in range(len(vetor_y)):
  caracter = saida[k]
  indice = dicionario_invertido[caracter]
  vetor_y[k][indice] = 1

for i in range(len(vetor_xBi)):# percorre as frases 
  for j in range(len(vetor_xBi[i])): #percorre cada caracter
    caracter = entradaBi[i][j]
    indice = dicionario_invertido[caracter]
    vetor_xBi[i][j][indice] = 1
    
for k in range(len(vetor_yBi)):
  caracter = saidaBi[k]
  indice = dicionario_invertido[caracter]
  vetor_yBi[k][indice] = 1

In [ ]:
modeloBi = Sequential()
modeloBi.add(LSTM(256, input_shape = (maxlen, len(spotteds_lista))))
modeloBi.add(Dense(len(spotteds_lista), activation='softmax'))


modelo = Sequential()
modelo.add(LSTM(256, input_shape = (maxlen, len(spotteds_lista))))
modelo.add(Dense(len(spotteds_lista), activation='softmax'))


modeloBi.compile(loss='categorical_crossentropy',optimizer='adam')

modelo.compile(loss='categorical_crossentropy',optimizer='adam')


In [ ]:
def amostra(neuronio, temp):
  neuronio_array = np.asarray(neuronio).astype('float64')
  neuronio_log = np.log(neuronio_array)/ temp
  neuronio_exp = np.exp(neuronio_log)
  neuronio_divisao = neuronio_exp/ np.sum(neuronio_exp)
  prob = np.random.multinomial(1, neuronio_divisao, 1)
  return np.argmax(prob)


def print_epoch(epoch):
  print(epoch)
  index = rd.randint(0, len(entrada))
  frase_atual = entrada[index]
  temperaturas = [0.2, 0.5, 1, 1.2]

  for temp in temperaturas:
    print(frase_atual, end = "")
    frase_aux = frase_atual
    for n_caracteres in range(50):
      vetor_x = np.zeros(1, (maxlen, len(spotteds_lista)), dtype=np.bool)
      for j in range(len(vetor_x[0])): #percorre cada caracter
        caracter = frase_aux[0][j]  #cria uma entrada binaria pra rede
        indice = dicionario_invertido[caracter]
        vetor_x[0][j][indice] = 1
        
      retorno = modelo.predict(vetor_x, verbose = 0)[0]
      indice_caracter = amostra(retorno, temp) #dada uma temperatura, recebemos o indice do proxmo caracter
      caracter_atual = dicionario_invertido[indice_caracter]
      print(caracter_atual, end="")
      frase_aux = frase_aux[1:]
      frase_aux = frase_aux + caracter_atual
  print()

#entrada -> Eu sou baralho
#entradaBi -> ohlarab uos uE

def print_epochBi(epoch):
  print(epoch)
  index = rd.randint(0, len(entradaBi))
  frase_atual = entradaBi[index]
  temperaturas = [0.2, 0.5, 1, 1.2]

  for temp in temperaturas:
    print(frase_atual, end = "")
    frase_aux = frase_atual
    for n_caracteres in range(50):
      vetor_x = np.zeros(1, (maxlen, len(spotteds_lista)), dtype=np.bool)
      for j in range(len(vetor_x[0])): #percorre cada caracter
        caracter = frase_aux[0][j]  #cria uma entrada binaria pra rede
        indice = dicionario_invertido[caracter]
        vetor_x[0][j][indice] = 1
        
      retorno = modeloBi.predict(vetor_x, verbose = 0)[0]
      indice_caracter = amostra(retorno, temp) #dada uma temperatura, recebemos o indice do proxmo caracter
      caracter_atual = dicionario_invertido[indice_caracter]
      print(caracter_atual, end="")
      frase_aux = frase_aux[1:]
      frase_aux = frase_aux + caracter_atual
  print()

In [ ]:
logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"


arquivo_caminho = 'pesos{epoch:02d}-{loss: .2f}.hdf5'

checkpoints = ModelCheckpoint(arquivo_caminho, monitor='loss', verbose=1, save_best_only=True, mode='min')


print_callback = LambdaCallback(on_epoch_end=print_epoch)
lista_callbacks = [checkpoints, print_callback]

modelo.fit(vetor_x, vetor_y, batch_size=128, epochs=25, callbacks=lista_callbacks)

modeloBi.fit(vetor_xBi, vetor_yBi, batch_size=128, epochs=25, callbacks=lista_callbacks)

In [ ]:
def testefinal(frase):
  print(frase, end="")
  frase_aux = frase
  frase_inv = frase[::-1]
  for n_caracteres in range(50):
    vetor_x = np.zeros(1, (maxlen, len(spotteds_lista)), dtype=np.bool)
    for j in range(len(vetor_x[0])): #percorre cada caracter
        caracter = frase_aux[0][j]  #cria uma entrada binaria pra rede
        indice = dicionario_invertido[caracter]
        vetor_x[0][j][indice] = 1
    retorno = modelo.predict(vetor_x, verbose = 0)[0]
    indice_caracter = amostra(retorno, temp)
    caracter_atual = dicionario_invertido[indice_caracter]
    print(caracter_atual, end="")
    frase_aux = frase_aux[1:]
    frase_aux = frase_aux + caracter_atual
  
    vetor_xBi = np.zeros(1, (maxlen, len(spotteds_lista)), dtype=np.bool)
    for j in range(len(vetor_xBi[0])): #percorre cada caracter
        caracter = frase_inv[0][j]  #cria uma entrada binaria pra rede
        indice = dicionario_invertido[caracter]
        vetor_xBi[0][j][indice] = 1
    retorno = modeloBi.predict(vetor_xBi, verbose = 0)[0]
    indice_caracter = amostra(retorno, temp)
    caracter_atual = dicionario_invertido[indice_caracter]
    frase_inv = frase_inv[1:]
    frase_inv = frase_inv + caracter_atual
    print(caracter_atual, end="")
